In [1]:
from tqdm import tqdm
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.llms.base import LLM 
from typing import Any,List,Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
from datasets import Dataset
from langchain.prompts import PromptTemplate
import os
from langchain.chains import RetrievalQA
import json, re

In [2]:
raw_data_path='../data/raw_data/'
persist_directory = '../data/rag_data/vector_db/chroma'
embedding_model_path = "/root/model/BAAI/"
embedding_model_model_kwargs = {"device": "cuda"}
embedding_model_encode_kwargs = {"normalize_embeddings": True}
llm_path='/root/model/internlm2-chat-7b'

In [3]:
## 判断是否有本地数据库
if not os.path.isdir(persist_directory):
    #加载文档和分
    loader = DirectoryLoader(raw_data_path,show_progress=True,use_multithreading=True)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=768,chunk_overlap=1, separators=["\n\n", "\n", " ", "","。","，"])
    split_docs = text_splitter.split_documents(docs)
    embeddings = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_path, model_kwargs=embedding_model_model_kwargs, encode_kwargs=embedding_model_encode_kwargs
    )

    # 加载数据库
    vectordb = Chroma.from_documents(
        documents=split_docs,
        embedding=embeddings,
        persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
    )
    # 将加载的向量数据库持久化到磁盘上
    vectordb.persist()

else:
    #加载已经有的向量数据库
    embeddings = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_path, model_kwargs=embedding_model_model_kwargs, encode_kwargs=embedding_model_encode_kwargs
    )
    vectordb = Chroma(persist_directory= persist_directory, embedding_function=embeddings)

In [4]:
# 测试一下向量数据库 用相似度来匹配相似的句子
query = "佟某驾驶一辆大客车（乘载54人，核载55人)行至太原境内以45公里的时速通过一处泥泞路段时，机动车侧滑驶出路外坠入深沟，导致14人死亡、40人受伤。佟某的主要违法行为是什么？"
docs = vectordb.similarity_search(query)
print(docs[0].page_content)

1354、佟某驾驶一辆大客车（乘载 54 人，核载 55 人）行至太原境内以 45 公里的 时速通过一处泥泞路段时，机动车侧滑驶出路外坠入深沟，导致 14 人死亡、40 人 受伤。本次事故佟某的主要违法行为是什么？

A、客车超员

B、超速行驶

C、酒后驾驶

D、疲劳驾驶

【答案】B

【技巧 1】复杂路段、特殊天气、特殊操作，限速 30；不得超速。

【技巧 2】解析：题中泥泞路段以 45 公里/小时行驶，属于超速行驶。避免侧滑， 泥泞道路最高行驶速度为 30 公里/小时。

【讲解 1】本题主要考察实际案例中对驾驶员违法行为的判断。按照规定，泥泞路 面最高速度不能超过 30 公里/小时，题中以 45 公里的时速通过一处泥泞路段，是 超速行驶。因此选择“超速行驶”。

【讲解 2】相关法规参考：《道路交通安全法实施条例》第四十六条，机动车行驶 中遇有下列情形之一的，最高行驶速度不得超过每小时 30 公里，其中拖拉机、电 瓶车、轮式专用机械车不得超过每小时 15 公里：（四）在冰雪、泥泞的道路上行 驶时。

1355、郝某驾驶一辆载有 84.84 吨货物的重型自卸货车（核载 15.58 吨），行至滦 县境内 262 省道 34 公里加 623 米处，与前方同向行驶的一辆载有 45.85 吨货物的 货车（核载 1.71 吨）追尾碰撞后，侧翻撞向路边人群，造成 19 人死亡、17 人受 伤。本次事故双方驾驶人共同的违法行为是什么？

A、超速行驶

B、货车超载

C、疲劳驾驶

D、酒后驾驶

【答案】B

【技巧 1】实载不得超过核载。


In [5]:
## 加载LLM
class InternLM_LLM(LLM):
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self,model_path:str):

        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True,attn_implementation="flash_attention_2",torch_dtype=torch.bfloat16).cuda()#
        self.model = self.model.eval()

    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        # 重写调用函数
        system_prompt = """你是交通法则小助手，熟知中华人民共和国公安部令和国务院令的交通法规知识，
        以及详尽的道路驾驶技能和安全文明常识考试内容。
        """
        
        messages = [(system_prompt, '')]
        response, history = self.model.chat(self.tokenizer, prompt , history=messages)
        return response
        
    @property
    def _llm_type(self) -> str:
        return "InternLM"
    
llm=InternLM_LLM(llm_path)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
# 定义一个 Prompt Template
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
                案。请提供详细而清晰的回答。确保回答涵盖相关法规和实际技能，尽量详细回答问题，并尽量避免简单带过问题。总是在回答的最后说“谢谢你的提问！”。
                {context}
                问题: {question}
                有用的回答:"""

custom_rag_prompt = PromptTemplate.from_template(template)
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [7]:
# 根据RAG的第一种写法
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, return_source_documents=True,
                                           chain_type_kwargs={"prompt": custom_rag_prompt})
qa_chain(query)

/root/.conda/envs/LLM/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': '佟某驾驶一辆大客车（乘载54人，核载55人)行至太原境内以45公里的时速通过一处泥泞路段时，机动车侧滑驶出路外坠入深沟，导致14人死亡、40人受伤。佟某的主要违法行为是什么？',
 'result': '佟某的主要违法行为是超速行驶。根据《中华人民共和国道路交通安全法》第四十二条的规定，驾驶人在通过泥泞路段时，应当按照道路条件降低行驶速度。而佟某在泥泞路段以每小时45公里的速度行驶，超过了泥泞路段的最高限速，导致了事故的发生。因此，佟某的主要违法行为是超速行驶。此外，车辆核载55人，但佟某驾驶的车辆载有54人，也存在超载的情况，但这并不是导致事故的主要原因。\n\n在驾驶车辆时，特别是在遇到复杂的路况和天气条件时，一定要遵守交通法规，严格控制车速，保障自身和他人的安全。 ',
 'source_documents': [Document(page_content='1354、佟某驾驶一辆大客车（乘载 54 人，核载 55 人）行至太原境内以 45 公里的 时速通过一处泥泞路段时，机动车侧滑驶出路外坠入深沟，导致 14 人死亡、40 人 受伤。本次事故佟某的主要违法行为是什么？\n\nA、客车超员\n\nB、超速行驶\n\nC、酒后驾驶\n\nD、疲劳驾驶\n\n【答案】B\n\n【技巧 1】复杂路段、特殊天气、特殊操作，限速 30；不得超速。\n\n【技巧 2】解析：题中泥泞路段以 45 公里/小时行驶，属于超速行驶。避免侧滑， 泥泞道路最高行驶速度为 30 公里/小时。\n\n【讲解 1】本题主要考察实际案例中对驾驶员违法行为的判断。按照规定，泥泞路 面最高速度不能超过 30 公里/小时，题中以 45 公里的时速通过一处泥泞路段，是 超速行驶。因此选择“超速行驶”。\n\n【讲解 2】相关法规参考：《道路交通安全法实施条例》第四十六条，机动车行驶 中遇有下列情形之一的，最高行驶速度不得超过每小时 30 公里，其中拖拉机、电 瓶车、轮式专用机械车不得超过每小时 15 公里：（四）在冰雪、泥泞的道路上行 驶时。\n\n1355、郝某驾驶一辆载有 84.84 吨货物的重型自卸货车（核载 15.58 吨），行至滦 县境内 262 省道 34 公里加 623 米处，与前方同向行驶的一辆载有 45.85 吨货物的 货车（核载 1.71 吨）追尾碰撞后，侧

In [8]:
# 根据RAG的第二种写法
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke(query)

'佟某的主要违法行为是超速行驶。根据交通法规，在泥泞路段行驶时，最高车速不得超过每小时30公里，而佟某以45公里的时速通过泥泞路段，违反了这一规定，导致了事故的发生。 '

In [33]:
# 把数据里面的query都RAG了  用于后面评测
eval_filename='../data/rag_data/eval/llm_conversation_dataset_v3.json'

# 评测数据集的格式
questions = []
ground_truth = []
answers = []
contexts = []

llm_answers=[]
with open(eval_filename, 'r', encoding='utf-8') as file:
    json_data=json.load(file)

In [ ]:
# 获取LLM原本的回答
cnt=50 #只取前面50个RAG
for item in tqdm(json_data):
    if cnt<0:break
    llm_answers.append({'llm_answers': llm(item['conversation'][0]['input'])})
    cnt-=1
# 将数据保存到文件
with open('../data/rag_data/eval/llm_answers.json', 'w',encoding='utf-8') as f:
    f.write(json.dumps(llm_answers))


In [34]:
# 获取RAG回答
cnt=50 #只取前面50个RAG
for item in tqdm(json_data):
    if cnt<0:break

    item_question=item['conversation'][0]['input']
    item_result=qa_chain(item_question)

    questions.append(item_question)
    answers.append(item_result['result'])
    contexts.append([item_result['source_documents'][0].page_content])
    ground_truth.append(item['conversation'][0]['output'])
    cnt-=1

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}
 
# Convert dict to dataset
dataset = Dataset.from_dict(data)
# 保存到本地
dataset.save_to_disk("../data/rag_data/eval/eval_dataset2")

  3%|▎         | 51/1596 [06:11<3:07:31,  7.28s/it]


Saving the dataset (0/1 shards):   0%|          | 0/51 [00:00<?, ? examples/s]